# Motifs ordering exploration

## Maria Inês Silva
## 01/02/2019

***

In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

# extendedMD code
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
import extendedMD.emd as emd
from extendedMD.viz import create_motif_table, plot_k_motifs
from extendedMD.pruning import prune_motifs_with_mdl, prune_motifs_with_dist

In [2]:
cwd = os.getcwd()
data_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'data'))
output_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'outputs/experiments_lat_acc'))

In [3]:
def plot_zoomin_motif(ts, motif_index, motif_point_list, events_ts,  x_lim):
    motif_pointers = motif_point_list[motif_index]
    event_df = pd.DataFrame([ts, events_ts.values]).T.reset_index()
    event_df.columns = ['index', 'var', 'event']
    fig = plt.figure(figsize=(25,10))
    plt.subplot(3,1,1)
    plt.plot(ts, 'tab:gray')
    for temp_point in motif_pointers:
        plt.plot(temp_point, ts[temp_point], 'tab:blue')
    sns.scatterplot(x="index", y="var", hue="event", data=event_df[event_df['event']>0], legend=False,
                    palette=sns.xkcd_palette(['red', 'tangerine', 'greenish yellow']))
    plt.ylim(min(ts), max(ts))
    plt.xlim(x_lim)
    return fig

## Data import

In [4]:
trip_df = pd.read_pickle(os.path.join(data_folder, 'trip_df.pkl'))
lat_acc_ts = trip_df['ay'].values
events_ts = trip_df['event_type'].values

In [5]:
file = os.path.join(output_folder, 'trip_exp4.p')
motif_dic_list = pickle.load(open(file, 'rb'))
R = 0.2

## Motif prunning

In [ ]:
start_time = time.time()
pruned_motif_dic_list = prune_motifs_with_mdl(lat_acc_ts, motif_dic_list, R)
pruned_motif_df = create_motif_table(pruned_motif_dic_list)
print("Prunning algorithm run in {} minutes".format(round((time.time() - start_time)/60, 2)))
pruned_motif_df

In [ ]:
plot_k_motifs(len(pruned_motif_df), lat_acc_ts, events_ts, pruned_motif_dic_list, 'lateral acceleration')

In [ ]:
start_time = time.time()
pruned_motif_dic_list = prune_motifs_with_dist(lat_acc_ts, motif_dic_list, R, 50)
pruned_motif_df = create_motif_table(pruned_motif_dic_list)
print("Prunning algorithm run in {} minutes".format(round((time.time() - start_time)/60, 2)))
pruned_motif_df

In [ ]:
plot_k_motifs(len(pruned_motif_df), lat_acc_ts, events_ts, pruned_motif_dic_list, 'lateral acceleration')

## Computing the events partition for each motif

In [6]:
def compute_event_partition(motif_dic_list, events_ts):
    motif_event_partition_list = []
    for motif_dic in motif_dic_list:
        motif_events_dic_list = []
        for member_pointers in motif_dic['members_ts_pointers']:
            member_events_list = [events_ts[i] for i in member_pointers]
            member_events_dic = {'brakes': np.sum([member_event==1 for member_event in member_events_list]),
                                 'turns': np.sum([member_event==2 for member_event in member_events_list]),
                                 'accelerations': np.sum([member_event==3 for member_event in member_events_list])}
            motif_events_dic_list.append(member_events_dic)
        motif_events_partition = pd.DataFrame(motif_events_dic_list).sum()
        if np.mean(motif_events_partition)==0:
            motif_events_partition['dispersion'] = 0
        else:
            motif_events_partition['dispersion'] = round(np.std(motif_events_partition)/float(np.mean(motif_events_partition)), 2)
        motif_event_partition_list.append(motif_events_partition)
    motif_partition_df = pd.DataFrame(motif_event_partition_list)
    return motif_partition_df

In [7]:
motif_partition_df = compute_event_partition(motif_dic_list, events_ts)
motif_partition_df.head()

,accelerations,brakes,turns,dispersion
0,0.0,0.0,2.0,1.41
1,0.0,0.0,0.0,0.00
2,5.0,1.0,1.0,0.81
3,0.0,0.0,0.0,0.00
4,0.0,0.0,0.0,0.00


## Create motif table

In [9]:
def create_motif_event_table(motif_dic_list, events_ts):
    pattern_list = [dic['pattern'] for dic in motif_dic_list]
    mdl_cost_list = [dic['mdl_cost'] for dic in motif_dic_list]
    mean_dist_list = [dic['mean_dist'] for dic in motif_dic_list]
    n_members_list = [len(dic['members_ts_pointers']) for dic in motif_dic_list]
    motif_table_dic = {
        'pattern': pattern_list,
        'pattern_len': [len(pattern) for pattern in pattern_list],
        'n_members': n_members_list,
        'mdl_cost': mdl_cost_list,
        'mean_dist': mean_dist_list
    }
    motif_partition_df = compute_event_partition(motif_point_list, events_ts)
    motif_df = pd.DataFrame(motif_dic) \
                        .join(motif_partition_df) \
                        .assign(mdl_cost = lambda x: x['mdl_cost'].round(1)) \
                        .sort_values('mdl_cost')
    return motif_df

In [11]:
motif_df = create_motif_event_table(motif_dic_list, events_ts)
motif_df.head()

NameError: name 'motif_point_list' is not defined

In [ ]:
motif_df.plot.scatter('n_members', 'dispersion')
plt.show()

In [ ]:
single_event_df = motif_df[motif_df['dispersion']>1.4]
len(single_event_df)

In [ ]:
temp = single_event_df['accelerations']+single_event_df['turns']+single_event_df['brakes']
temp.describe()

In [ ]:
single_event_df.plot.scatter('mdl_cost', 'mean_dist')
plt.show()

In [ ]:
max_brake = np.sum([event==1 for event in events_ts])
max_turn = np.sum([event==2 for event in events_ts])
max_acc = np.sum([event==3 for event in events_ts])
print(max_brake, max_turn, max_acc)

In [ ]:
motif_df.sort_values('accelerations', ascending=False).head(10)

In [ ]:
motif_df.sort_values('brakes', ascending=False).head(10)

In [ ]:
single_event_df.sort_values('turns', ascending=False).head(10)

In [ ]:
plot_zoomin_motif(ts_1d, 317, motif_point_list, events_ts,  [800, 2500])
plt.show()

In [ ]:
plot_zoomin_motif(ts_1d, 317, motif_point_list, events_ts,  [3000, 5000])
plt.show()

In [ ]:
# extendedMD code
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from extendedMD.motif_viz import plot_single_motif

plot_single_motif(ts_1d, events_ts, 317, motif_point_list, motif_center_list, pattern_list, 'lateral acc')
plt.show()

In [ ]:
plot_single_motif(ts_1d, events_ts, 37, motif_point_list, motif_center_list, pattern_list, 'pca')
plt.show()